In [1]:
import math

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import pythainlp

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

from data_tokenizer import load_corpus

from model.new_sdc import NewSDC
from model.sdc import SDC

### Load Data

In [29]:
file_name = 'ผู้บริโภค - TescoLotus'

corpus, labels = load_corpus('../data/' + file_name + '.txt')

len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

f = open('../data/tokenized/tokenized_' + file_name + '.txt')
tokenized_corpus = eval(f.read())
f.close()

Total documents 268
1 clusters


### Preprocess Corpus

#### Remove Words

In [30]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.extend(['นี้'])
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 1449 words
filter frequent words: 605 words
filter letter words: 604 words
filter stop words: 403 words


In [31]:
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [32]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

#### Doc2Vec

In [33]:
tagged_corpus = [TaggedDocument(doc, [i]) for i, doc in enumerate(idx_corpus)]
model = Doc2Vec(tagged_corpus, vector_size=average_doc_size, window=4, min_count=2, epochs=100)
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

paragraph_vectors = [model.infer_vector(doc) for doc in idx_corpus]
paragraph_vectors = pandas.DataFrame(paragraph_vectors, dtype=float)

### Clustering

In [34]:
def get_onehot(corpus, weight):
    dictionary = Dictionary(corpus)
#     dictionary.filter_extremes(no_below=2, no_above=1, keep_n=len(dictionary))

    bow_corpus = [dictionary.doc2bow(doc) for doc in corpus]
    if weight == 'normal':
        weight_corpus = bow_corpus
    elif weight == 'tfidf':
        tfidf = TfidfModel(bow_corpus, smartirs='ltc')
        weight_corpus = [tfidf[doc] for doc in bow_corpus]

    unique_words = [dictionary[id] for id in range(len(dictionary))]
    array = numpy.zeros((len(corpus), len(unique_words)), dtype=float)
    for i, doc in enumerate(weight_corpus):
        for id, score in doc:
            array[i, id] = score

        if weight == 'normal' and len(doc) != 0:
#             array[i] = numpy.divide(array[i], len(idx_corpus[i]))
            array[i] = numpy.divide(array[i], len(doc))
    
    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [35]:
def generate_result(predicted_labels, marks):
    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = idx_corpus
    result['label'] = labels
    result['predicted_label'] = predicted_labels
    if marks:
        result['marks'] = marks
    else:
        result['marks'] = -1
    return result

In [36]:
def eval_cluster(onehot_corpus, result):
    label_count = numpy.unique(result['predicted_label'])
    num_cluster = label_count[-1] + 1

    clusters = [[] for i in range(num_cluster)]
    corpus_centroid = []
    for i, label in result['predicted_label'].iteritems():
        clusters[label].append(numpy.array(onehot_corpus.iloc[i]))
        corpus_centroid.append(numpy.array(onehot_corpus.iloc[i]))
    corpus_centroid = numpy.mean(corpus_centroid, axis=0).reshape(1, -1)   

#     print('\tIntra cluster sim\tInter cluster sim\tIntra / Inter')
    compactness = 0
    centroids = []
    for i in range(num_cluster):
        size = len(clusters[i])
        if size != 0:
            centroid = numpy.mean(clusters[i], axis=0)
            centroids.append(centroid)
            centroid = centroid.reshape(1, -1)
            similarities = cosine_similarity(centroid, clusters[i])
            compactness += numpy.sum(similarities)

#             intra = numpy.sum(similarities) / size
#             inter = cosine_similarity(centroid, corpus_centroid)[0][0]
#             print(i, end='\t')
#             print(intra, end='\t')
#             print(inter, end='\t')
#             print(intra / inter)
    return compactness, centroids

In [37]:
min_samples = 7
eps = 0.32
expand_rate = 0.05
epoch = 15

onehot_corpus = get_onehot(idx_corpus, 'normal')
# onehot_corpus = get_onehot(filtered_corpus, 'tfidf')

In [11]:
max_compactness = 0
for i in range(epoch):
    model = NewSDC()
    _tpredicted_labels, marks = model.predict(onehot_corpus, min_samples, eps, expand_rate)

#     model = SDC()
#     _tpredicted_labels, marks = model.predict(onehot_corpus, min_samples, eps, expand_rate)
    
#     marks = None
    
#     model = DBSCAN(metric='cosine', eps=eps, min_samples=min_samples).fit(onehot_corpus)
#     _tpredicted_labels = model.labels_ + 1

#     model = KMeans(n_clusters=14).fit(onehot_corpus)
#     _tpredicted_labels = model.labels_
    
    _tresult = generate_result(_tpredicted_labels, marks)
    compactness, _tcentroids = eval_cluster(onehot_corpus, _tresult)
    
    if compactness > max_compactness:
        max_compactness = compactness
        predicted_labels = _tpredicted_labels
        result = _tresult
        centroids = _tcentroids
        
print(max_compactness)
label_count = numpy.unique(result['predicted_label'], return_counts=True)[1]
num_cluster = len(label_count)
print(label_count)

196.09866662549476
[51 13 47 19  7 15 12 10 19 24 26  8 10 14 30 10 11  7 10  7]


In [ ]:
centroids = None 
prev_label_count = None
while True:
    model = NewSDC()
    predicted_labels, marks = model.predict(onehot_corpus, min_samples, eps, expand_rate, seeds=centroids)
    
    result = generate_result(predicted_labels, marks)
    compactness, centroids = eval_cluster(onehot_corpus, result)
    
    label_count = numpy.unique(result['predicted_label'], return_counts=True)[1]
    if numpy.array_equal(label_count, prev_label_count):
        break
    prev_label_count = label_count
    centroids = centroids[1:]
    
    print(compactness)
    print(label_count)
num_cluster = len(label_count)

In [12]:
sims = cosine_similarity(centroids)
new_labels = [i for i in range(num_cluster)]
print(new_labels)
for i, row in reversed(list(enumerate(sims))):
    for j, value in reversed(list(enumerate(row[:i + 1]))):
        if i != j and value >= eps - eps / 20:
            print(i, j, value)
            base = min(new_labels[i], new_labels[j])
            new_labels[j] = base
            new_labels = [base if label == new_labels[i] else label for label in new_labels]
print(new_labels)

grouped_labels = numpy.zeros(len_corpus)
for i, label in enumerate(predicted_labels):
    grouped_labels[i] = new_labels[label]
new_result = generate_result(grouped_labels, None)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
17 14 0.3469191623179261
14 13 0.35747313686377363
13 11 0.7625572195447531
10 4 0.34222424468593116
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 4, 11, 12, 11, 11, 15, 16, 11, 18, 19]


### Result

In [13]:
class Widget:
    def __init__(self, result, column_name):
        self.result = result
        self.column_name = column_name
        
        label_count = numpy.unique(result['predicted_label'])
        self.widget = widgets.ToggleButtons(
            options=[int(num) for num in label_count],
            disabled=False,
            button_style='',
        )
        
        self.widget.observe(self.on_click, names='index')
        self.on_click({'new' : 0})
        
    def on_click(self, change):
        clear_output()
        display(self.widget)
        new = self.widget.options[change['new']]
        for index, value in self.result[self.result['predicted_label'] == new].iterrows():
            if value['marks'] == 0:
                print("@", end="")
            elif value['marks'] == 1:
                print("*", end="")
            print(index, value[self.column_name])

In [38]:
# result.to_csv('../data/results/new_sdc/' + file_name + '.csv')

result = pandas.read_csv('../data/results/new_sdc_em/' + file_name + '.csv')

print(eval_cluster(onehot_corpus, result)[0])
print(numpy.unique(result['predicted_label'], return_counts=True)[1])

120.90348922297676
[72 30 75 20 16 16 11  8 12  8]


In [39]:
w1 = Widget(new_result, 'comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 15, 16, 18, 19), value=0)

4 ซื้อลิขสิทธิ์ UCL มาด้วยนะ จ่ายรายเดือนไป แต่ไม่มีให้ดู
17 Wi-Fi ทรูที่บ้าน ใช้ได้แต่กลางคืน กลางวันใช้ไม่ได้ แต่จ่ายพันกว่าบาททุกเดือน ควรปรับปรุง ฝากทรูคอฟฟี่ไปบอกด้วย
21 ทำไมเอาเมนูทวิซที่ปั่นใส่ขนมช็อกโกแลตทวิซคาลาเมลออก 💥🔥🔥🔥โกรธหนักค่ะ เพราะชอบมากกกกกกกก
28 ขอทำขนมไปส่งได้ไหมคะ นักเรียนบ้านยากจนจะสอนนักเรียนแต่ไม่มีที่ให้โอกาสนักเรียนคะ แต่รับรองขนมอร่อยคะ
42 ตัวอักษรใหญ่ๆ มองเมนูไม่ชัด ไม่กล้าสั่ง กลัวสั่งผิดละเขินน 555+
45 บอกพนักงงานชงกาแฟทำหน้าตาดีๆ เลิกกระแทกแก้วสักที เวลาส่งคืนลูกค้าเยอะๆอ่ะ ละก็ช่วยทำคอมเม้นลูกค้าให้มันเป็นตามที่ขอด้วยอิเวน
52 ตอนปรับโปรใหม่เดือนเเรกเนตอย่างเเรง ผ่านไปสักพัก ช้าโคตร ส่วนกาเเฟงั้นๆ ราคาเเพงไป
54 พนง.น่ารักๆ เยอะนะ แต่เพราะชื่อ ทรู นี่แหละตรูเลยไม่เข้า 5555
57 เป็นร้านที่แดกกาแฟร้อนทีไร แม่งต้องล้วกลิ้นไหม้เป็นแผลตลอด น้ำร้อนเอ็งจะร้อนไปไหนวะ สงสัยแม่งเอามาจากบ่อในนรกชัวร์ๆ ถ้าเอ็งร้อนแบบนี้
70 Iced Black Tea Latte หาชงอร่อยยากมาก อยากกินทีต้องลุ้นทีว่ามือชงจะทำได้แค่ไหน เข้าร้านทรูเพื่อการนี้โดยเฉพาะ เมื่อไม่เจอมือเซียนชง ก็ไม่เข้าไปเหยียบ

In [40]:
w2 = Widget(result, 'comment')

ToggleButtons(index=3, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=3)

*3 คิว เปิดช่องเยอะๆหน่อยค่ะ คนไหนใช้ช่องทางด่วน ผิดกติกา จะยอมช่วยเห็นใจคนทำถูกกติกาด้วยจ้า
*10 ทุกคนล้วนแล้ว แต่มีหน้าที่ของเขา คนเราควรให้โอกาสซึ่งกันและกันนะครับให้คิดว่าคนดีก็มี คนไม่ดีก็มีนะครับ ผมไม่ได้โลกสวยนะครับ แต่ผมก็ทำงานในด้านบริการเหมือนกัน ถ้าไม่มายืนตรงจุดที่เขายืนคุณก็จะไม่รู้ คิดบวกนะครับ พนักงานเหมือนกันนะครับ ปรับปรุงตรงไหนแล้วดี ก็ควรที่ต้องปรับปรุงแก้ไขส่วนที่ผิดพลาดของตัวเรา ดีแล้วยังดีได้อีกนะครับ Mc.Tom
*17 เทสโก้โลตัสสาขาบางกะปิทีวีตัวนี้อยู่บริเวณหน้าร้าน Treephaphan Spa ชั้นลอย(ชั้นเดียวกับไปรษณีย์)ใกล้ประตูออกลานจอดรถวันนั้นผมเดินชนเต็มๆ และผมเชื่อว่าต้องมีลูกค้าท่านอื่นเดินชนทีวีตัวนี้มาแล้วหลายครั้ง มันไม่ควรอยู่ตรงนี้นะครับเพราะนี่คือทางเดิน และมันก็ไม่ได้ชิดเข้ามุมกำแพงเลย เดินเลี้ยวขวาจากทีวีตัวนี้ไปก็ลงไปชั้นG ซึ่งมันเป็นทางหลักที่คนเข็นรถลงจากซุปเปอร์ฯใช้ประจำนะครับ ไม่ใช่ทางลับที่ไม่มีคนเดินผ่าน เพดานชั้นนี้ก็ค่อนข้างต่ำอยู่แล้วแถมเอาทีวีตัวนี้มาติด ต้องคนที่มีความสูง150ลงไปเท่านั้นแหล่ะครับที่จะเดินลอดผ่านได้ (ผมสูง175)ผมเชื่อว่าทางตัวเทสโก้โลตัสย

In [25]:
w3 = Widget(result, 'tokenized_comment')

ToggleButtons(index=17, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18), value=17)

*20 ['เหมือน', 'สาขา', 'รสชาติ', 'เหมือน', 'คน', 'ทำ', 'กาแฟ', 'รสชาติ', 'ลูกค้า', 'ทรู', 'สิทธิ์', 'ลด', 'พิเศษ']
*71 ['ปรับปรุง', 'เรื่อง', 'สัญญาน']
*100 ['ปรับปรุง', 'สัญญาน', 'มือถือ']
*164 ['รสชาติ', 'งั้น']
*166 ['สิทธิ์', 'รสชาติ', 'งั้น']
*180 ['รู้', 'ปรับปรุง', 'แดก']
*198 ['ปรับปรุง']
*200 ['ปรับปรุง', 'เรื่อง', 'อินเตอร์เน็ต']
@234 ['ปรับปรุง', 'รสชาติ']
*273 ['กาแฟ', 'รสชาติ', 'เหมือน']
*313 ['ปรับปรุง', 'เวลา', 'นั่ง', 'ร้าน']


In [18]:
seed = 0
compare = 0

a = numpy.array(onehot_corpus.iloc[seed]).reshape(1, -1)
b = numpy.array(onehot_corpus.iloc[compare]).reshape(1, -1)
print(cosine_similarity(a,b))

print(idx_corpus[seed])
print(corpus[seed])
print(idx_corpus[compare])
print(corpus[compare])

[[1.]]
['โลตัส', 'สาขา', 'จ้าง', 'พนักงาน', 'พนักงาน', 'แผนก', 'คน', 'หน้าที่', 'ตัว', 'รับผิดชอบ', 'แคชเชียร์', 'งาน', 'แผนก', 'ตัว', 'เละ', 'โดน', 'ด่า', 'โดน', 'ด่า', 'เครียด', 'หน้า', 'ใส่', 'ลูกค้า', 'ทำ', 'งาน', 'บริการ', 'ลูกค้า', 'ดี', 'ลูกค้า', 'รัก', 'แนะนำ', 'ใจ', 'เด็ก', 'โลตัส', 'เก่า']
โลตัสไฮเปอร์มาเก็ต​ ทุกสาขา​ ควรจ้างพนักงานเยอะกว่านี้​ ไม่ควรเรียกพนักงานแผนกอื่นไปลงเครื่องแคชเชียร์​บ่อยจนเกินไป​ ทุกคนต้องมีหน้าที่ที่ตัวเองต้องรับผิดชอบ​ พอช่วยแคชเชียร์เสร็จ​ งานแผนกตัวเองเละก็โดนด่า​ พอโดนด่าก็เกิดความเครียด​ หน้าบึ้งตึงใส่ลูกค้า​ ทำงานไม่มีความสุข​ บริการลูกค้าไม่ดี​ แล้วลูกค้าก็ร้องเรียน​ เพราะรักจึงอยากแนะนำ​ จากใจเด็กโลตัสเก่า​
['โลตัส', 'สาขา', 'จ้าง', 'พนักงาน', 'พนักงาน', 'แผนก', 'คน', 'หน้าที่', 'ตัว', 'รับผิดชอบ', 'แคชเชียร์', 'งาน', 'แผนก', 'ตัว', 'เละ', 'โดน', 'ด่า', 'โดน', 'ด่า', 'เครียด', 'หน้า', 'ใส่', 'ลูกค้า', 'ทำ', 'งาน', 'บริการ', 'ลูกค้า', 'ดี', 'ลูกค้า', 'รัก', 'แนะนำ', 'ใจ', 'เด็ก', 'โลตัส', 'เก่า']
โลตัสไฮเปอร์มาเก็ต​ ทุกสาขา​ ควรจ้างพนักงานเยอะ